In [1]:
include("src/pkgs.jl")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
include("src/riot.jl")
using Main.Riot

In [ ]:
leagues = ["challenger"]

function scrape_match(id)
    mdata = load_match(id)
    mdata
end

function scrape_summoner(data)
    id = data["summonerId"]
    sdata = load_summoner(id)
    puuid = sdata["puuid"]
    matches = load_matches_for(puuid)
    map(scrape_match, matches)
end

function scrape_league(l)
    ldata = load_league(l)
    map(scrape_summoner, ldata["entries"])
end

map(scrape_league, leagues)

Loading league challenger
Loading summoner hRHUp3pDrtCJM6BNSzEMiWtu-s7Obnj5mEHihE91WQTnwiBgSu0F_hqb-A
Loading matches for u-iAax7P0b1uAzMVVKdp26UP2bHHSEwiAFWbYN1cn_BXCttTnNCvM7QULximTMxPJccIdvn8j_VCdQ
Loading match NA1_4315424340
Loading match NA1_4314660077
Loading match NA1_4314584278
Loading match NA1_4313740637
Loading match NA1_4313693923
Loading match NA1_4312819570
Loading match NA1_4312773201
Loading match NA1_4312260241
Loading match NA1_4312223763
Loading match NA1_4312181310
Loading match NA1_4311533468
Loading match NA1_4311476659
Loading match NA1_4311098758
Loading match NA1_4311084690
Loading match NA1_4311060852
Loading match NA1_4310658855
Loading match NA1_4310662927
Loading match NA1_4310624077
Loading match NA1_4310547503
Loading match NA1_4309364187
Loading summoner w7OocV5OYyKL1z_fXUpD3i3KU3NbjGDjsSzcJVFbo8jEs4o
Loading matches for zbkRKBOGJ2wnDaFIgrHbA1pMs4XaJ5Y3nKYkKHtIvyO8C8QGAgX7kGbb0cWzikONL7xW04GmgBHyxg
Loading match NA1_4320805706
Loading match NA1_43198515